In [2]:
import os
import json
import re
from IPython.display import display, HTML
from pprint import pprint
import copy

In [39]:
wiki_path = "/data/ephemeral/home/jhyeop/level2-mrc-nlp-11/data/wikipedia_documents.json"
if os.path.exists(wiki_path):
    with open(wiki_path, "r", encoding="utf-8") as f:
        wiki_data = json.load(f)

In [40]:
# 태그형 문자열
def find_taglike_words(text):
    pattern = r'<(?![^<>]*[가-힣])[^<>]+>'
    matches = re.findall(pattern, text)
    
    return matches

In [41]:
taglike_words_set = set()
num_docs_with_taglike_words = 0
num_taglike_words = 0

for document in wiki_data.values():
    taglike_words = find_taglike_words(document["text"])
    if taglike_words:
        num_docs_with_taglike_words += 1
        taglike_words_set.update(taglike_words)
        num_taglike_words += len(taglike_words)


print(f"태그 형식의 문자열 총 개수: {num_taglike_words}")
print(f"태그 형식 문자열을 포함한 문서의 수: {num_docs_with_taglike_words}")
print(f"태그 형식 문자열 종류: {len(list(taglike_words_set))}")
print(f"태그 형식 문자열 예시: {list(taglike_words_set)[:10]}")

태그 형식의 문자열 총 개수: 3200
태그 형식 문자열을 포함한 문서의 수: 966
태그 형식 문자열 종류: 963
태그 형식 문자열 예시: ['<Shiny Happy People>', '<mapper namespace="org.mybatis.example.BlogMapper">', '<Satellites>', '<blockquote class="toccolours" style="margin:1px 1px; padding: 10px 15px 10px 15px; display:table; {}">', '<Dear Air>', '<t.rex>', '<Everybody Wants to Rule the World>', '<Curiga>', '<Shock Value>', '<!DOCTYPE HTML PUBLIC\n     "-//W3C//DTD HTML 4.01 Transitional//EN"\n     "http://www.w3.org/TR/html4/loose.dtd">']


In [42]:
# openai API를 이용해 추출한 태그 형식 문자열의 카테고리 (HTML_TAG, CODE_REPRESENTATION, PROPER_NOUN)
taglike_words_category_path = "/data/ephemeral/home/jaehyeop/level2-mrc-nlp-11/data/default/categorized_taglike_words.json"
with open(taglike_words_category_path, "r", encoding="utf-8") as f:
    taglike_words_category = json.load(f)

### 코드 문서 제거

In [43]:
# 전처리된 문서 모음
normalized_wiki = copy.deepcopy(wiki_data)
print(f"최초 문서 수: {len(normalized_wiki.keys())}")

최초 문서 수: 60613


코드 태그 (ex. < iostream> 이런 식으로 생긴 문자열) 포함한 문서 중 삭제해도 무방하다고 판단되는 것 제거

In [44]:
code_set = set()
for taglike_word, category in taglike_words_category.items():
    if category == "CODE_REPRESENTATION":
        for doc_id, doc in normalized_wiki.items():
            if taglike_word in doc["text"]:
                code_set.add(doc_id)

# 수기로 검증하여 의미가 있다고 판단한 문서(문서 전체가 코드로 도배되어 있지 않은 문서)
for idx in ["43445", "34273", "13921", "59622", "15290", "55000", "28801", "41935"]:
    code_set.remove(idx)
code_list = list(code_set)
for doc_id in code_list:
    del normalized_wiki[doc_id]

print(f"코드 문서 {len(code_list)}개 제거 ==> wiki 문서 수: {len(normalized_wiki.keys())}")
print("===제거 목록===")
print(code_list)

코드 문서 32개 제거 ==> wiki 문서 수: 60581
===제거 목록===
['54048', '875', '42977', '1163', '44841', '5717', '44644', '36187', '57300', '46913', '5429', '21402', '9626', '41822', '43231', '13231', '54049', '21404', '24405', '11950', '52868', '52870', '48074', '31950', '37660', '37232', '56438', '27917', '33245', '42734', '16071', '50606']


코드에 자주 등장하는 문자열을 이용해 문서를 찾고, 그 중 삭제할 것, 전처리할 것을 분류

In [45]:
code_set2 = set()
num_docs_with_code2 = 0
code_pattern = r'(\+=|\\frac|return |stdio|[^C]\+\+| int |def |const )'
for doc in normalized_wiki.values():
    codelike = re.findall(code_pattern, doc["text"])
    if codelike:
        print(doc)
        print(codelike)
        code_set2.update(codelike)
        num_docs_with_code2 += 1
print(f"코드가 포함된 문서가 {num_docs_with_code2}개 더 존재합니다.")

{'text': '2세대 언어는 기계어 최상단에 추상적인 수준을 제공한다. TX-0, PDP-1과 같은 컴퓨터들을 이용하여 코딩하던 과거에는 MIT 해커들이 했던 최초의 일은 어셈블러를 작성하는 일이었다. \n\n대부분의 어셈블러는 매크로를 제공하므로 공통이 되는 일련의 명령들을 만들어낼 수 있다.\n\n아래는 상단의 피보나치 수 계산기와 동일하지만, MASM 문법을 이용하여 x86 어셈블리어로 표현한 것이다:\n\nfib:\n    mov edx, [esp+8]\n    cmp edx, 0\n    ja @f\n    mov eax, 0\n    ret\n\n    @@:\n    cmp edx, 2\n    ja @f\n    mov eax, 1\n    ret\n\n    @@:\n    push ebx\n    mov ebx, 1\n    mov ecx, 1\n\n    @@:\n        lea eax, [ebx+ecx]\n        cmp edx, 3\n        jbe @f\n        mov ebx, ecx\n        mov ecx, eax\n        dec edx\n    jmp @b\n\n    @@:\n    pop ebx\n    ret\n\n\n동일 기능을 C로 나타내면 다음과 같다:\n\nunsigned int fib(unsigned int n)\n{\n    if (n <= 0)\n        return 0;\n    else if (n <= 2)\n        return 1;\n    else {\n        unsigned int a,b,c;\n        a = 1;\n        b = 1;\n        while (1) {\n            c = a + b;\n            if (n <= 3) return c;\n            a = b;\n            b = c;\n            n--;\n        }\n    }\n}', 'corpus

In [46]:
# 문서 삭제 또는 전처리가 필요한 문서를 수기 분류
code_delete = ["1164", "1254", "1927", "2638", "4152", "5718", "5808", "6481", "7192", "8706", "9193", "12104", "12700", "12702", "12703", "12704", "15291", "16070", "17096", "17097", "21403", "25097", "32382", "37378", "45570", "45705", "46851", "48850", "51530", "55062", "55063", "55064", "55065", "55066", "55068", "55069", "55070", "59622", "49452", "54545", "23451", "60", "26706"]
code_preprocess = ["362", "1255", "3868", "4916", "5809", "7394", "8422", "9384", "11668", "15292", "34273", "34292", "41663", "45189", "49013", "50502", "50992", "54440", "59474", "59689", "59690", "13630"]


In [47]:
for code_id in code_delete:
    normalized_wiki.pop(code_id, None)

print(f"코드 문서 {len(code_delete)}개 더 제거 ==> wiki 문서 수: {len(normalized_wiki.keys())}")

코드 문서 43개 더 제거 ==> wiki 문서 수: 60538


In [48]:
docs_with_code_preprocessed_path = "/data/ephemeral/home/jaehyeop/level2-mrc-nlp-11/data/documents_removed_code.json"
with open(docs_with_code_preprocessed_path, "r", encoding="utf-8") as f:
    docs_with_code_preprocessed = json.load(f)

for doc_id, preprocessed_text in docs_with_code_preprocessed.items():
    normalized_wiki[doc_id]["text"] = preprocessed_text["text"]

print(f"{len(code_preprocess)}개의 문서에서 코드 표현을 전처리하였습니다.")

22개의 문서에서 코드 표현을 전처리하였습니다.


### HTML TAG 제거

In [49]:
num_docs_with_html_tag = 0
html_tag_set = set()
num_html_tag = 0

for taglike_word, category in taglike_words_category.items():
    if category == "HTML_TAG":
        html_tag_set.add(taglike_word)                         
        for doc_id, doc in normalized_wiki.items():
            if taglike_word in doc["text"]:
                num_docs_with_html_tag += 1
                processed_text = doc["text"]
                tags = re.findall(taglike_word, processed_text)
                num_html_tag += len(tags)
                processed_text = re.sub(taglike_word, " ", processed_text)
                normalized_wiki[doc_id]["text"] = processed_text


print(f"{num_docs_with_html_tag}개의 문서에서 총 {num_html_tag}개의 HTML 태그를 제거하였습니다.")
print(f"HTML 태그 종류 개수: {len(list(html_tag_set))}")
print(f"HTML 태그 Ex) {list(html_tag_set)[:5]}")

746개의 문서에서 총 1995개의 HTML 태그를 제거하였습니다.
HTML 태그 종류 개수: 195
HTML 태그 Ex) ['<mapper namespace="org.mybatis.example.BlogMapper">', '<blockquote class="toccolours" style="margin:1px 1px; padding: 10px 15px 10px 15px; display:table; {}">', '<!-- Web Service offering endpoints for both the bindings-->', '<operation ref="tns:Put" />', '<xs:element name="body" type="xs:anyType" minOccurs="0"/>']


### 페이지 인용 표시 제거

In [50]:
# 페이지 인용 표시 찾기
num_docs_with_page_ref = 0
page_ref_set = set()
num_page_ref = 0
for doc_id, doc in normalized_wiki.items():
    page_refs = re.findall(r'(?:p{1,2}|pages?)=[0-9]+(?:[–-][0-9]+)?', doc["text"])
    if not page_refs:
        continue
    num_page_ref += len(page_refs)
    processed_text = doc["text"]
    for page_ref in page_refs:
        # 연도 표시와 붙어있는 경우 3가지 존재 -> 직접 문자열 수정
        if page_ref == "p=3031818":
            page_ref = "p=303"
        elif page_ref == "pages=443–4452015":
            page_ref = "pages=443–445"
        elif page_ref == "p=451938":
            page_ref = "p=45"
        page_ref_set.add(page_ref)
        processed_text = re.sub(r'(?:p{1,2}|pages?)=[0-9]+(?:[–-][0-9]+)?', " ", processed_text)
    num_docs_with_page_ref += 1
    normalized_wiki[doc_id]["text"] = processed_text


print(f"{num_docs_with_page_ref}개의 문서에서 총 {num_page_ref}개의 페이지 인용 표시를 제거하였습니다.")
print(f"페이지 인용 표시 종류 개수: {len(list(page_ref_set))}")
print(f"페이지 인용 표시 Ex) {list(page_ref_set)[:5]}")

674개의 문서에서 총 2888개의 페이지 인용 표시를 제거하였습니다.
페이지 인용 표시 종류 개수: 1023
페이지 인용 표시 Ex) ['p=287', 'p=11', 'pp=94–96', 'pp=13-14', 'p=440']


### URL 제거

In [51]:
# URL 제거
num_docs_with_urls = 0
num_urls = 0
url_set = set()
url_pattern = r"https?:/?/?[a-zA-Z0-9.-]+(?:\.[a-zA-Z]{2,})(?:/[^\s]*)?"
for doc_id, doc in normalized_wiki.items():
    urls = re.findall(url_pattern, doc["text"])
    if urls:
        num_docs_with_urls += 1
        num_urls += len(urls)
        url_set.update(urls)
        processed_text = doc["text"]
        processed_text = re.sub(url_pattern, " ", processed_text)
        normalized_wiki[doc_id]["text"] = processed_text


print(f"{num_docs_with_urls}개의 문서에서 총 {num_urls}개의 url을 제거하였습니다.")
print(f"url 종류 개수: {len(list(url_set))}")
print(f"url Ex)")
pprint(list(url_set)[:5])

221개의 문서에서 총 431개의 url을 제거하였습니다.
url 종류 개수: 350
url Ex)
['http://www.theatlantic.com/doc/199302/kaplan|periodical=The',
 'http://www.seoul-marina.com/marina/16/',
 'http://www.zimfree.com',
 'http://www.kcomics.net/Artist/view_info.asp?cdidx=3827',
 'https://web.archive.org/web/20160401162611/http://apfelboymchen.net/gnu/pronunciation/']


### loc 표시 제거

In [52]:
def find_loc_ref(text):
    refs = []
    refs.extend(re.findall(r'loc=[0-9]+ and [0-9]+', text))
    text = re.sub(r'loc=[0-9]+ and [0-9]+', " ", text)

    refs.extend(re.findall(r'loc=§ [0-9]+ and [0-9]+', text))
    text = re.sub(r'loc=§ [0-9]+ and [0-9]+', " ", text)

    refs.extend(re.findall(r'loc=[0-9]+', text))
    text = re.sub(r'loc=[0-9]+', " ", text)

    refs.extend(re.findall(r'loc=§ [0-9]+', text))
    text = re.sub(r'loc=§ [0-9]+', " ", text)

    refs.extend(re.findall(r'loc=[a-zA-Z0-9 ]+', text))
    text = re.sub(r'loc=[a-zA-Z0-9 ]+', " ", text)

    refs.extend(re.findall(r'loc=[^ .]*', text))
    text = re.sub(r'loc=[^ .]*', " ", text)
    return text, refs


num_locs = 0
num_docs_with_locs = 0
loc_set = set()
for doc_id, doc in normalized_wiki.items():
    processed_text = doc["text"]
    processed_text, locs = find_loc_ref(processed_text)
    if locs:
        num_docs_with_locs += 1
        num_locs += len(locs)
        loc_set.update(locs)
        normalized_wiki[doc_id]["text"] = processed_text


print(f"{num_docs_with_locs}개의 문서에서 총 {num_locs}개의 loc 표시를 제거하였습니다.")
print(f"loc 표시 종류 개수: {len(list(loc_set))}")
print(f"loc 표시 Ex) {list(loc_set)[:5]}")


634개의 문서에서 총 2948개의 loc 표시를 제거하였습니다.
loc 표시 종류 개수: 488
loc 표시 Ex) ['loc="別境心所(별경심소)"', 'loc=§ 13', 'loc=제20권', 'loc="法處所攝色(법처소섭색)"', 'loc="五位"']


### 주석 문구 제거

In [53]:
# | string ... }} 형태 주석 문구 찾기
num_docs_with_annotations = 0
num_annotations = 0
annotation_set = set()
for doc_id, doc in normalized_wiki.items():
    annotations = re.findall(r'\|[\s\S]*?\}\}', doc["text"])
    if annotations:
        processed_text = doc["text"]
        annotation_set.update(annotations)
        num_docs_with_annotations += 1
        num_annotations += len(annotations)
        processed_text = re.sub(r'\|[\s\S]*?\}\}', " ", processed_text)
        normalized_wiki[doc_id]["text"] = processed_text


print(f"{num_docs_with_annotations}개의 문서에서 총 {num_annotations}개의 주석을 제거하였습니다.")
print(f"주석 문구 종류 개수: {len(list(annotation_set))}")
print(f"주석 문구 Ex) {list(annotation_set)[:5]}")


577개의 문서에서 총 1209개의 주석을 제거하였습니다.
주석 문구 종류 개수: 993
주석 문구 Ex) ['|T.1613| . T31n1613_p0852b04 - T31n1613_p0852b06. 5정진(五精進)|ps"云何精進。謂懈怠對治。善品現前。勤勇為性。謂若被甲。若加行。若無怯弱。若不退轉。若無喜足。是如此義。圓滿成就。善法為業。"\n\n무착의 《현양성교론》에 따르면, 불교 경전에서는 유세(有勢) 유근(有勤) 유용(有勇) 견맹(堅猛) 불사선액(不捨善軛)이라 명명하고 있다.무착 조, 현장 한역|T.1602| . p. T31n1602_p0481c08 - T31n1602_p0481c11. 정진(精進)|ps= 심소는 마음이 용맹하여 게으름이 없으며, 스스로 가볍고 천하게 하지 않음을 체성으로 삼는다. 해태(懈怠)의 장애를 끊음을 업으로 삼고, 앞에서와 같이 나아가서 정진을 증장함을 업으로 삼는다. 경전에서 “정진을 일으켜서 힘이 있음 용감함 견고하고 용맹스러움에 머물러서 선(善)의 멍에를 버리지 않는다”고 말한 바와 같다.\n 정진(精進, vīrya) 심소는 ‘정진’의 능력으로서 근(勤)이라고도 하며 부른다. 용맹스럽게 선행을 닦고 악행을 끊게 하는 심리작용이다. 해태(懈怠) 심소를 다스린다.\n 소[牛]에게 멍에를 씌움으로써 소로 하여금 도망가지 않고 나아가며 충실히 일하게 한다. 선법(善法)도 역시 그러해서 수행자에게 멍에를 씌워 선품(善品)에서 벗어나지 않고 열반에 나아가게 하므로 이렇게 표현한다. 선행은 비록 실천하기 어렵지만 수행자는 중도에 포기하지 않고 감당해나가야 하는 것임을 나타낸다."}}', '|Tug of War was a number-one album in both the UK and the US.  〈Ebony and Ivory〉는 빌보드 100에서 1위를 차지한 매카트니의 28번째 싱글이다.  다음 해, 그와 잭슨은 〈Say Say Say〉를 작업했고, 이는   매카트니가 가장 최근에 미국 1위에 오른 것이고, 그해에 나온 그의 음반의 타이틀곡 

In [57]:
# 기타 삭제, 전처리 필요한 문서 처리

need_normalize = ["1945", "1824", "2016", "7527", "28172", "37746", "39067"]
for doc_id in need_normalize:
    if doc_id in normalized_wiki.keys():
        normalized_wiki[doc_id]["text"] = re.sub(r'[^ ]*\|[^|]+\|(?:[^|]+\|)*[^ \n]*', " ", normalized_wiki[doc_id]["text"])

need_remove = ["973", "1667", "1970", "3780", "3879", "4379", "5527", "6221", "8334", "8433", "8933", "9335", "9693", "10156", "11949", "13192", "14412", "22519", "26045", "32847", "39948", "40095", "41224", "43406", "56572", "59707"]
for doc_id in need_remove:
    normalized_wiki.pop(doc_id, None)

# | string | string | ... 형태의 주석은 패턴화할 수 없어 openai API를 활용하여 전처리하였음
with open("/data/ephemeral/home/jaehyeop/level2-mrc-nlp-11/data/documents_removed_annotation.json", "r", encoding="utf-8") as f:
    docs_removed_annotations = json.load(f)

for doc_id, preprocessed_doc in docs_removed_annotations.items():
    if doc_id in normalized_wiki.keys():
        normalized_wiki[doc_id]["text"] = preprocessed_doc["text"]

print(f"{len(need_normalize) + len(need_remove) + len(docs_removed_annotations.keys())}개의 문서가 추가적으로 전처리되었습니다.")


2159개의 문서가 추가적으로 전처리되었습니다.


In [58]:
print(f"최종 전처리된 문서 개수: {len(normalized_wiki.keys())}")

최종 전처리된 문서 개수: 60512


In [59]:
normalized_wikipedia_documents_path = "/data/ephemeral/home/jaehyeop/level2-mrc-nlp-11/data/normalized_wikipedia_documents.json"
with open(normalized_wikipedia_documents_path, "w", encoding="utf-8") as writer:
        writer.write(json.dumps(normalized_wiki, indent=4, ensure_ascii=False) + "\n")

In [ ]:
# |string| 형태의 주석 문구 찾기
num_docs_with_annotation_type2 = 0
annotation_type2_set = set()
annotation_dict = {}
for doc_id, doc in normalized_wiki.items():
    annotations = re.findall(r'[^ ]*\|[^|]+\|(?:[^|]+\|)*[^ ]*', doc["text"])
    if not annotations:
        continue
    print(doc)
    print(annotations)
    annotation_type2_set.update(annotations)
    num_docs_with_annotation_type2 += 1
    annotation_dict[doc_id] = doc["text"]

annotation_documents = "/data/ephemeral/home/jaehyeop/level2-mrc-nlp-11/data/documents_with_annotation.json"
with open(annotation_documents, "w", encoding="utf-8") as writer:
        writer.write(json.dumps(annotation_dict, indent=4, ensure_ascii=False) + "\n")

print(f"주석 문구가 포함된 문서 수: {num_docs_with_annotation_type2}")
print(f"주석 문구 개수(중복 제거): {len(list(annotation_type2_set))}")
print(f"주석 문구 Ex) {list(annotation_type2_set)[:5]}")